# Practicum 04 - Dimensionality Reduction and Clustering with Structured Biomedical Data

In this practicum, we apply _Principal Components Analysis (PCA)_ and _Hierarchical Clustering_ to the NCI60 cancer cell line obtained from the [ISLP repository](https://github.com/intro-stat-learning/ISLP). This dataset contains microarray data that represents 6830 gene expression level measurements for each of 64 cancer cell lines representing 14 different types of cancer.  __Our goal is examine this data using PCA and clustering.__. We will develop a PCA and Clustering model using [scikit-learn](https://scikit-learn.org/stable/index.html).

Before working with the NCI60 data, we will first illustrate PCA and Hierachcial Clustering using a simulated dataset.

In [ ]:
# imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans, AgglomerativeClustering as HClust
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree
import numpy as np

# local project imports
import sys
sys.path.append('../../src') # alternatively add to path using: pip install -e /path/to/src
from plotting import plt_box_grid_by_target, plt_box_grid, plt_xy_scatter_grid
from cluster_utils import plot_hclust
from util import load_data

In [ ]:
# global settings
pd.options.display.max_columns = 100
rs = 654321 # random state, use this to ensure reproducibility

# Simulated data

To get started, let's generate simulated data for classification. Here, we will craete a dataset with __many more features than samples__. This is representative of many genomics experiments where we may measure expression levels of many or all of the more than 20,000 human genes in a relatively smaller population. We will use this data to illustrate PCA and Hierarchical Clustering. We will also demonstrate how PCA can be used to transform the data and improve classification model results on the test data.

To generate the simulated data, we use the `make_classification` method from [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html#). For demonstration, we will split the data into training and test sets.

In [ ]:
# generate simulated classification data
n = 200 # number of samples
n_train = 150
n_features = 1000
n_classes = 4
X, y = make_classification(n_samples=n, n_features=n_features, n_informative=25, n_redundant=0,
                           n_classes=n_classes, n_clusters_per_class=1, 
                           flip_y=0.01, class_sep=1.0, hypercube=True, weights=None,
                           shift=0.0, scale=1.0, shuffle=True, random_state=rs)

# split into train and test
X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]

# PCA

## Simulated Data

Let's start by applying the PCA to the simulated data. We use the scikit-learn [PCA module][https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html]. To obtain the PCA transformation of the data, we apply the `PCA.fit_transform` method to the training data, `X_train`.

In [ ]:
# fit the PCA model
sim_pca = PCA()
sim_train_scores = sim_pca.fit_transform(X_train)

Although the data may have up to $min\left(n-1, p\right)$, where here $n=n_train=150$ and $p=n_features=1000$, we will see that much of the variance is accounted for by a smaller number of principal components and that we can achieve higher classifier performance with a small number of principal components as compared to using all the features.

First, let's plot the data using the first two principal components and then using the first and third principal components. To do so, we use the `sim_train_scores` variable which contains the transformation of the original data, `X_train` to the principal components using a linear combination of the original features where the coeficients are the _loading factors_ for each original feature for the given principal component.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
ax = axes[0]
ax.scatter(sim_train_scores[:, 0], sim_train_scores[:, 1], c=y_train, cmap='viridis', alpha=0.5)
ax.set_xlabel('PC1'); ax.set_ylabel('PC2')

ax = axes[1]
ax.scatter(sim_train_scores[:, 0], sim_train_scores[:, 2], c=y_train, cmap='viridis', alpha=0.5)
ax.set_xlabel('PC1'); ax.set_ylabel('PC3')

The plot markers are colored by their class membership. Even though this data was created with a 1000 features, we can see that in the principal component space samples in the same class tend to cluster together.

To get a better sense of the propportion of variance explained (PVE), we can plot the PVE as a function of the principal components. This is known as a a __scree plot__. We can also plot the cumulative sum of the PVE as a function of the principal component index. From the _scree plot_, we see that there is an inflection near the $15^{th}$ principal component indicating that subsequent principal components explain much less of the variance than the first 15 principal components. Additionally, the plot of the cumulative PVE indicates that about 50% of the variance is explained by the first 50 principal components, while the remaining 100 components are needed to account for the remaining variance.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
ax = axes[0]

ticks = np.arange(sim_pca.n_components_)+1
ax.plot(ticks,
        sim_pca.explained_variance_ratio_,
        marker='x')
ax.set_xlim(0,50)
ax.set_xlabel('Principal Component')
ax.set_ylabel('Proportion Variance Explained (PVE)')

ax = axes[1]
ax.plot(ticks,
        sim_pca.explained_variance_ratio_.cumsum(),
        marker='x')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Cumulative Proportion Variance Explained (PVE)')

Finally, we can compare the performance of a logistic regression classifier trained to predict class membership using all of the features (with regularization) vs. one trained on data transformed to scores using only the first 15 principal components. We expect that, on the test set, the model trainined with the principal components score features will perform better. Caution, this is only a single experiment and is not representative of every case.

In [ ]:
clf_all = LogisticRegression(random_state=rs).fit(X_train, y_train)
sim_test_scores = sim_pca.transform(X_test)
n_components = 20
sim_test_scores.shape
clf_pca = LogisticRegression(random_state=rs).fit(sim_train_scores[:, :n_components], y_train)

print(classification_report(y_test, clf_all.predict(X_test)))

print(classification_report(y_test, clf_pca.predict(sim_test_scores[:, :n_components])))

# PCA on Cancer Gene Expression Data

Now let's apply PCA to the NCI60 cancer gene expression dataset. First, we load the data and transform it using the scikit-learn StandardScalar.

In [ ]:
# load the NCBI gene expression data 
n = 150 # number of samples
n_train = 100
n_features = 1000
n_classes = 4
X_nci, y_nci = load_data('NCI60', "../source_data")
scaler = StandardScaler()
X_nci_scaled = scaler.fit_transform(X_nci)
color = y_nci['label'].astype('category').cat.codes
y_nci['color'] = color

# Problem 1 (1 point)

Apply PCA to the NCI60 feature data contained in the `X_nci_scaled` variable.

In [ ]:
###### YOUR CODE HERE
nci_pca = None
nci_scores = None

# Problem 2 (1 point)

Plot the NCI60 data using the principal component scores contained in the `nci_scores` variable. Create two plots:
1. The transformed data using the first and third principal component scores, `nci_scores[:,0]` and `nci_scores[:,1]`
2. The transformed data using the first and third principal component scores, `nci_scores[:,0]` and `nci_scores[:,2]`

In both cases, set the color argument to `c=y_nci['color']`, which is an integer coresponding to the cancer type for that sample.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
colors = y_nci['color']

###### YOUR CODE HERE

# Problem 3 (1 point)
Here, create two plots illustrating the variance explained by the principal components for the NCI60 data using the `nci_pca` variable:
1. The _Proportion of Variance (PVE)_ for the principal components
2. The _Cummulative PVE_ for the principal components

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

###### YOUR CODE HERE

# Clustering

## Simulated Data

Now let's examine how we can create a _dendrogram_ that results from Hierarchical Clustering using the [scikit learn AgglomerativeClustering module](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering). To create the dendrogram, we use the method `plot_hclust` in this repository under _src/cluster_utils.py_. This method requires the data to be clustered, here we use `X_train`, and string value specifying the linkage function to be used, here we consider _Complete_ and _Average_, and a figure axis.

The results of this clustering will not be particularly informative for the simulated data. However, this illustrates the method for use on the NCI60 data.

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(15, 20))
sim_labels = y_train
ax = axes[0]; hc_comp = plot_hclust(X_train, sim_labels, 'Complete', ax)
ax = axes[1]; hc_comp = plot_hclust(X_train, sim_labels, 'Average', ax)

# Clustering on Cancer Gene Expression Data

Now, let's apply hierarchical clustering to the NCI60 gene expression data. First, let's reload the data.

In [ ]:
# load the NCBI gene expression data 
n = 150 # number of samples
n_train = 100
n_features = 1000
n_classes = 4
X_nci, y_nci = load_data('NCI60', "../source_data")
scaler = StandardScaler()
X_nci_scaled = scaler.fit_transform(X_nci)
color = y_nci['label'].astype('category').cat.codes
y_nci['color'] = color

# Problem 4 (1 point)

Using the `plot_clust` method, plot the dendrogram for the `X_nci_scaled` data. The labels for this data are in `y_nci['label']`. Create two plots:
1. Dendrogram using the _Complete_ linkage function
2. Dendrogram using the _Average_ linkage function

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(15, 20))

###### YOUR CODE HERE

# Problem 5 (2 points)
What is your interpretation of the NCI60 dendrograms? Consider at what height you might make a cut to form clusters. Approximatly, how many clusters result from making a cut above 100? How pure are the clusters with respect to the cancer types represented in them? 

YOUR RESPONSE HERE